Thanks to https://github.com/YuanGongND/ssast

In [ ]:
# !pip download timm===0.4.5 -d ./timm/
# !pip download torchinfo -d ./torchinfo/

In [ ]:
!mkdir ./timm
!mkdir ./ssastbasepatch400
!cp ../input/birdclef2022-ssast/timm/timm-0.4.5-py3-none-any.whl ./timm/timm-0.4.5-py3-none-any.whl
!cp ../input/birdclef2022-ssast/ssastbasepatch400/SSAST-Base-Patch-400.pth ./ssastbasepatch400/SSAST-Base-Patch-400.pth

In [ ]:
# !pip install ../input/birdclef2022-ssast/timm/Pillow-9.1.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# !pip install ../input/birdclef2022-ssast/timm/certifi-2021.10.8-py2.py3-none-any.whl
# !pip install ../input/birdclef2022-ssast/timm/charset_normalizer-2.0.12-py3-none-any.whl
# !pip install ../input/birdclef2022-ssast/timm/idna-3.3-py3-none-any.whl
# !pip install ../input/birdclef2022-ssast/timm/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
# !pip install ../input/birdclef2022-ssast/timm/requests-2.27.1-py2.py3-none-any.whl
!pip install ./timm/timm-0.4.5-py3-none-any.whl
# !pip install ../input/birdclef2022-ssast/timm/torch-1.11.0-cp37-cp37m-manylinux1_x86_64.whl
# !pip install ../input/birdclef2022-ssast/timm/torchvision-0.12.0-cp37-cp37m-manylinux1_x86_64.whl
# !pip install ../input/birdclef2022-ssast/timm/typing_extensions-4.2.0-py3-none-any.whl
# !pip install ../input/birdclef2022-ssast/timm/urllib3-1.26.9-py2.py3-none-any.whl
# !pip install ../input/birdclef2022-ssast/torchinfo/torchinfo-1.6.5-py3-none-any.whl

In [ ]:
# !pip install timm===0.4.5
# !pip install torchinfo

In [ ]:
import torch.nn as nn
import torch
import torchaudio
from torchaudio import transforms
import sys
import os
from timm.models.layers import trunc_normal_
import timm
import numpy as np
from timm.models.layers import to_2tuple
from random import randrange
import pandas as pd
from matplotlib import pyplot as plt
import random
from IPython.display import Audio, display
from ast import literal_eval
# from torchinfo import summary

#from tqdm import tqdm
from collections import defaultdict

In [ ]:
# from os.path import exists
# import requests

# pretrained_mdl_path = './ssastbasepatch400/SSAST-Base-Patch-400.pth'
# os.makedirs(os.path.dirname('./ssastbasepatch400/'), exist_ok=True)
# file_exists = exists(pretrained_mdl_path)

# if file_exists == False:
#     url = 'https://www.dropbox.com/s/ewrzpco95n9jdz6/SSAST-Base-Patch-400.pth?dl=1'

#     r = requests.get(url, allow_redirects=True)

#     open(pretrained_mdl_path, 'wb').write(r.content)

pretrained_mdl_path = "../input/birdclef2022-ssast/ssastbasepatch400/SSAST-Base-Patch-400.pth"


In [ ]:
# sys.path.append("/data/sls/scratch/yuangong/aed-trans/src/models/")
# sys.path.append("/data/sls/scratch/yuangong/aed-trans/src/")

In [ ]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
root_dir = '../input/birdclef-2022'
df = pd.read_csv(root_dir + '/train_metadata.csv')
df.head()

In [ ]:
columns_of_interest = ['primary_label', 'filename', 'secondary_labels', 'rating']
df = df[columns_of_interest]
df.head()

In [ ]:
# import json

# with open('../input/birdclef-2022/scored_birds.json') as sbfile:
#     scored_birds = json.load(sbfile)
#     df = df[df['primary_label'].isin(scored_birds)]

In [ ]:
labels = df.primary_label.unique().tolist()
label2id = {labels[i]: i for i in range(len(labels))}

In [ ]:
train_audio_dir = '../input/birdclef-2022/train_audio'

In [ ]:
# for i in tqdm(range(len(df))):
#     filename = df.loc[i,'filename']
#     sig, sr = torchaudio.load(os.path.join(train_audio_dir, filename))
#     df.loc[i, 'num_channels'] = sig.shape[0]  # number of audio channels (mono/stereo)
#     df.loc[i, 'signal_len'] = sig.shape[1]  # signal length
#     df.loc[i, 'sampling_rate'] = sr
#     df.loc[i, 'duration'] = sig.shape[1] // sr

In [ ]:
# os.mkdir('./audio_16k/')
# for i in tqdm(range(len(train_df))):
#     filename = train_df.loc[i,'filename']
#     print('sox ' + train_audio_dir + '/audio/' + filename + ' -r 16000 ' + './audio_16k/' + filename)
#     os.system('sox ' + train_audio_dir + '/audio/' + filename + ' -r 16000 ' + './audio_16k/' + filename)

In [ ]:
# df['num_channels'] = df['num_channels'].astype('int64')
# df['signal_len'] = df['signal_len'].astype('int64')
# df['sampling_rate'] = df['sampling_rate'].astype('int64')
# df.head()

In [ ]:
# print(df['duration'].max(), df['duration'].min())

In [ ]:
def train_test_split(df, frac=0.2):
    
    # get random sample 
    test = df.sample(frac=frac, axis=0, random_state=333)

    # get everything but the test sample
    train = df.drop(index=test.index)
    
    train.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)

    return train, test

In [ ]:
train_data, test_data = train_test_split(df, frac=0.1)
#train_data = df

# DataLoader

In [ ]:
class AudioDataset():
    def __init__(self, audio_conf, pd_data, label2id, audio_dir):
        """
        Dataset that manages audio recordings
        :param audio_conf: Dictionary containing the audio loading and preprocessing settings
        :param dataset_json_file
        """
        self.pd_data = pd_data
        self.data = pd_data.to_dict('index')
        self.audio_conf = audio_conf
        self.audio_dir = audio_dir
        
        
        self.melbins = self.audio_conf.get('num_mel_bins')
        self.freqm = self.audio_conf.get('freqm')
        self.timem = self.audio_conf.get('timem')
        self.mixup = self.audio_conf.get('mixup')
        self.secondary_label = self.audio_conf.get('secondary_label')
        
        # dataset spectrogram mean and std, used to normalize the input
        self.norm_mean = self.audio_conf.get('mean')
        self.norm_std = self.audio_conf.get('std')
        # skip_norm is a flag that if you want to skip normalization to compute the normalization stats using src/get_norm_stats.py, if Ture, input normalization will be skipped for correctly calculating the stats.
        # set it as True ONLY when you are getting the normalization stats.
        self.skip_norm = self.audio_conf.get('skip_norm') if self.audio_conf.get('skip_norm') else False
        if self.skip_norm:
            print('now skip normalization (use it ONLY when you are computing the normalization stats).')
        else:
            print('use dataset mean {:.3f} and std {:.3f} to normalize the input.'.format(self.norm_mean, self.norm_std))
        # if add noise for data augmentation
        self.noise = self.audio_conf.get('noise')
        if self.noise == True:
            print('now use noise augmentation')

        self.index_dict = label2id
        self.label_num = len(self.index_dict)
        print('number of classes is {:d}'.format(self.label_num))
        
    def _monoToStereo(self, aud):
        sig, sr = aud
        if sig.shape[0] == 2:
            return aud
        else:
            stereo_sig = torch.cat([sig, sig])

        return (stereo_sig, sr)
    
    def _stereoToMono(self, aud):
        sig, sr = aud
        if sig.shape[0] == 1:
            return aud
        else:
            mono_sig = torch.mean(sig, dim=0, keepdim=True)
        
        return (mono_sig, sr)
    
    def _channelsNumber(self, aud, num_channel=1):
#         sig, sr = aud
#         transform = torchaudio.transforms.Resample(sr, 16*1000)
#         sig = transform(sig)
#         aud = (sig, 16*1000)
        if num_channel == 1:
            return self._stereoToMono(aud)
        else:
            return self._monoToStereo(aud)

    def _wav2fbank(self, filename, filename2=None):
        # mixup
        path1 = os.path.join(self.audio_dir, filename)
        if filename2 == None:
            aud = torchaudio.load(path1)
            waveform, sr = self._channelsNumber(aud)
            waveform = waveform - waveform.mean()
        # mixup
        else:
            aud1 = torchaudio.load(os.path.join(self.audio_dir, filename))
            aud2 = torchaudio.load(os.path.join(self.audio_dir, filename2))
            waveform1, sr = self._channelsNumber(aud1)
            waveform2, _ = self._channelsNumber(aud2)

            waveform1 = waveform1 - waveform1.mean()
            waveform2 = waveform2 - waveform2.mean()

            if waveform1.shape[1] != waveform2.shape[1]:
                if waveform1.shape[1] > waveform2.shape[1]:
                    # padding
                    p = waveform1.shape[1] - waveform2.shape[1]
                    l = random.randint(0, p)
                    temp_wav = torch.zeros(1, waveform1.shape[1])
                    temp_wav[0, l:(waveform2.shape[1] + l)] = waveform2
                    waveform2 = temp_wav
                else:
                    # cutting
                    p = waveform2.shape[1] - waveform1.shape[1]
                    l = random.randint(0, p)
                    waveform2 = waveform2[0, l:(waveform1.shape[1] + l)]

            # sample lambda from uniform distribution
            #mix_lambda = random.random()
            # sample lambda from beta distribtion
            mix_lambda = np.random.beta(10, 10)

            mix_waveform = mix_lambda * waveform1 + (1 - mix_lambda) * waveform2
            waveform = mix_waveform - mix_waveform.mean()

        fbank = torchaudio.compliance.kaldi.fbank(waveform, htk_compat=True, sample_frequency=sr, use_energy=False,
                                                  window_type='hanning', num_mel_bins=self.melbins, dither=0.0, frame_shift=10)

        target_length = self.audio_conf.get('target_length')
        n_frames = fbank.shape[0]

        p = target_length - n_frames

        # cut and pad
        if p > 0:
            l = random.randint(0, p)
            m = torch.nn.ZeroPad2d((0, 0, l, p - l))
            fbank = m(fbank)
        elif p < 0:
            k = -p
            j = random.randint(0, k)
            fbank = fbank[j:j+target_length, :]

        if filename2 == None:
            return fbank, 0
        else:
            return fbank, mix_lambda

    def __getitem__(self, index):
        """
        returns: image, audio, nframes
        where image is a FloatTensor of size (3, H, W)
        audio is a FloatTensor of size (N_freq, N_frames) for spectrogram, or (N_frames) for waveform
        nframes is an integer
        """
        # do mix-up for this sample (controlled by the given mixup rate)
        secon_lab = 0.5
        if random.random() < self.mixup:
            datum = self.data[index]
            # find another sample to mix, also do balance sampling
            # sample the other sample from the multinomial distribution, will make the performance worse
            # mix_sample_idx = np.random.choice(len(self.data), p=self.sample_weight_file)
            # sample the other sample from the uniform distribution
            mix_sample_idx = random.randint(0, len(self.data)-1)
            mix_datum = self.data[mix_sample_idx]
            
            score_coe = 0.5 if datum['rating'] <= 2.5 else 1
            mix_score_coe = 0.5 if mix_datum['rating'] <= 2.5 else 1
            # get the mixed fbank
            fbank, mix_lambda = self._wav2fbank(datum['filename'], mix_datum['filename'])
            # initialize the label
            label_indices = np.zeros(self.label_num)
            # add sample 1 labels
            label_indices[int(self.index_dict[datum['primary_label']])] += mix_lambda * score_coe
            if self.secondary_label:
                for secondary_label in literal_eval(datum['secondary_labels']):
                    label_indices[int(self.index_dict[secondary_label])] = mix_lambda * secon_lab * score_coe
            # add sample 2 labels
            label_indices[int(self.index_dict[mix_datum['primary_label']])] += (1.0-mix_lambda) * mix_score_coe
            if self.secondary_label:
                for secondary_label in literal_eval(mix_datum['secondary_labels']):
                    label_indices[int(self.index_dict[secondary_label])] = (1.0-mix_lambda) * secon_lab * mix_score_coe
            
            label_indices = torch.FloatTensor(label_indices)
        # if not do mixup
        else:
            datum = self.data[index]
            score_coe = 0.5 if datum['rating'] <= 2.5 else 1
            
            label_indices = np.zeros(self.label_num)
            fbank, mix_lambda = self._wav2fbank(datum['filename'])
            label_indices[int(self.index_dict[datum['primary_label']])] = 1.0 * score_coe
            if self.secondary_label:
                for secondary_label in literal_eval(datum['secondary_labels']):
                    label_indices[int(self.index_dict[secondary_label])] = secon_lab * score_coe

            label_indices = torch.FloatTensor(label_indices)


        # normalize the input for both training and test
        if not self.skip_norm:
            fbank = (fbank - self.norm_mean) / (self.norm_std * 2)
        # skip normalization the input if you are trying to get the normalization stats.
        else:
            pass
        
        if random.random() < 0.5:
            if self.noise == True:
                fbank = fbank + torch.rand(fbank.shape[0], fbank.shape[1]) * np.random.rand() / 10
                fbank = torch.roll(fbank, np.random.randint(-10, 10), 0)

        # the output fbank shape is [time_frame_num, frequency_bins], e.g., [1024, 128]
        return fbank, label_indices

    def __len__(self):
        return len(self.pd_data)

In [ ]:
target_length = 500
num_mel_bins = 128
batch_size = 12
pin_memory = True

In [ ]:
# set skip_norm as True only when you are computing the normalization stats
# audio_conf = {'num_mel_bins': num_mel_bins, 'target_length': target_length, 'freqm': 0, 'timem': 0, 'mixup': 0.75, 'skip_norm': True, 'mode': 'train'}
# ad = AudioDataset(audio_conf, train_data, label2id, train_audio_dir)       

# train_loader = torch.utils.data.DataLoader(ad, batch_size=256, shuffle=True, num_workers=0, pin_memory=True)
# mean=[]
# std=[]
# for i in range(3):
#     for (audio_input, labels) in train_loader:
#         cur_mean = torch.mean(audio_input)
#         cur_std = torch.std(audio_input)
#         mean.append(cur_mean)
#         std.append(cur_std)
#     print(i+1)

# print(np.mean(mean), np.mean(std))

In [ ]:
# import seaborn as sns

# sns.kdeplot([f for f in frames if f < 10000], x="x")
# # Add labels
# plt.title('Histogram of Frames')
# plt.xlabel('x')
# plt.ylabel('y')

In [ ]:
# sampler_rate 32k
mean = -6.5556197 #-4.8866115 #-3.6730156
std = 3.9615002 #4.4127903 #4.336685

# sampler_rate 16k
# mean = -7.348659
# std = 4.028582

In [ ]:
audio_conf = {'num_mel_bins': num_mel_bins, 'target_length': target_length, 'freqm': 0, 'timem': 0, 'mixup': 0.5,
              'mode': 'train', 'mean': mean, 'std': std, 'noise': True, 'secondary_label': True}

val_audio_conf = {'num_mel_bins': num_mel_bins, 'target_length': target_length, 'freqm': 0, 'timem': 0, 'mixup': 0,
                  'mode': 'evaluation', 'mean': mean, 'std': std, 'noise': False, 'secondary_label': False}

In [ ]:
train_audioset = AudioDataset(audio_conf, train_data, label2id, train_audio_dir)
test_audioset = AudioDataset(val_audio_conf, test_data, label2id, train_audio_dir)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_audioset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=pin_memory)
test_dataloader = torch.utils.data.DataLoader(test_audioset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=pin_memory)

# AST model

In [ ]:
# override the timm package to relax the input shape constraint.
class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x

def get_sinusoid_encoding(n_position, d_hid):
    ''' Sinusoid position encoding table '''

    def get_position_angle_vec(position):
        return [position / np.power(10000, 2 * (hid_j // 2) / d_hid) for hid_j in range(d_hid)]

    sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(n_position)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1

    return torch.FloatTensor(sinusoid_table).unsqueeze(0)

class ASTModel(nn.Module):
    def __init__(self, label_dim=527,
                 fshape=128, tshape=2, fstride=128, tstride=2,
                 input_fdim=128, input_tdim=1024, model_size='base',
                 pretrain_stage=True, load_pretrained_mdl_path=None):

        super(ASTModel, self).__init__()
        assert timm.__version__ == '0.4.5', 'Please use timm == 0.4.5, the code might not be compatible with newer versions.'
        
        # override timm input shape restriction
        timm.models.vision_transformer.PatchEmbed = PatchEmbed
        
        # pretrain the AST models
        if pretrain_stage == True:
            if load_pretrained_mdl_path != None:
                raise ValueError('Setting load_pretrained_mdl_path at pretraining stage is useless, pretraining is always from scratch, please change it to None.')
            if fstride != fshape or tstride != tshape:
                raise ValueError('fstride != fshape or tstride != tshape, they must be same at the pretraining stage, patch split overlapping is not supported.')

            # if AudioSet pretraining is not used (but ImageNet pretraining may still apply)
            if model_size == 'tiny':
                self.v = timm.create_model('vit_deit_tiny_distilled_patch16_224', pretrained=False)
                self.heads, self.depth = 3, 12
                self.cls_token_num = 2
            elif model_size == 'small':
                self.v = timm.create_model('vit_deit_small_distilled_patch16_224', pretrained=False)
                self.heads, self.depth = 6, 12
                self.cls_token_num = 2
            elif model_size == 'base':
                self.v = timm.create_model('vit_deit_base_distilled_patch16_384', pretrained=False)
                self.heads, self.depth = 12, 12
                self.cls_token_num = 2
            elif model_size == 'base_nokd':
                self.v = timm.create_model('vit_deit_base_patch16_384', pretrained=False)
                self.heads, self.depth = 12, 12
                self.cls_token_num = 1
            else:
                raise Exception('Model size must be one of tiny, small, base, base_nokd')

            self.original_num_patches = self.v.patch_embed.num_patches
            self.oringal_hw = int(self.original_num_patches ** 0.5)
            self.original_embedding_dim = self.v.pos_embed.shape[2]

            # SSL Pretraining Code
            self.softmax = nn.Softmax(dim=-1)
            self.lsoftmax = nn.LogSoftmax(dim=-1)
            self.fshape, self.tshape = fshape, tshape
            self.fstride, self.tstride = fstride, tstride
            self.input_fdim, self.input_tdim = input_fdim, input_tdim
            # this is a trick to make state_dict to track pretraining input_fdim and input_tdim and save them by using torch.save
            self.p_input_fdim, self.p_input_tdim = nn.Parameter(torch.tensor(input_fdim), requires_grad=False), nn.Parameter(torch.tensor(input_tdim), requires_grad=False)

            # masked patch classification (discriminative objective) layer
            # we use two layers for pretext task, but using a single layer has similar performance.
            # we map the output of transformer (768-dim for base models) to 256-dim patch input space, and then dot product with flattened patch input (also 256-dim) to calculate loss.
            # alternatively, you can map the output of transformer to 768-dim patch embedding space, and dot product with patch embedding. Performance-wise they are similar, but map to 256 space is more efficient.
            self.cpredlayer = nn.Sequential(nn.Linear(self.original_embedding_dim, self.original_embedding_dim), nn.ReLU(), nn.Linear(self.original_embedding_dim, 256))
            # masked patch reconstruction (generative objective) layer
            self.gpredlayer = nn.Sequential(nn.Linear(self.original_embedding_dim, self.original_embedding_dim), nn.ReLU(), nn.Linear(self.original_embedding_dim, 256))
            self.unfold = torch.nn.Unfold(kernel_size=(fshape, tshape), stride=(fstride, tstride))

            # we use learnable mask embedding (follow the BEIT paper), but using a fixed mask embedding (e.g., 0) leads to same performance.
            self.mask_embed = nn.Parameter(torch.zeros([1, 1, self.original_embedding_dim]))
            self.mask_embed = torch.nn.init.xavier_normal_(self.mask_embed)

            # get the intermediate shape
            self.p_f_dim, self.p_t_dim = self.get_shape(fstride, tstride, input_fdim, input_tdim, fshape, tshape)
            num_patches = self.p_f_dim * self.p_t_dim
            self.num_patches = num_patches
            self.v.patch_embed.num_patches = num_patches
            print('pretraining patch split stride: frequency={:d}, time={:d}'.format(fstride, tstride))
            print('pretraining patch shape: frequency={:d}, time={:d}'.format(fshape, tshape))
            print('pretraining patch array dimension: frequency={:d}, time={:d}'.format(self.p_f_dim, self.p_t_dim))
            print('pretraining number of patches={:d}'.format(num_patches))

            # the linear patch projection layer, use 1 channel for spectrogram rather than the original 3 channels for RGB images.
            new_proj = torch.nn.Conv2d(1, self.original_embedding_dim, kernel_size=(fshape, tshape), stride=(fstride, tstride))
            self.v.patch_embed.proj = new_proj

            # use trainable positional embedding
            new_pos_embed = nn.Parameter(torch.zeros(1, self.v.patch_embed.num_patches + self.cls_token_num, self.original_embedding_dim))
            self.v.pos_embed = new_pos_embed
            trunc_normal_(self.v.pos_embed, std=.02)

        # use a pretrained models for finetuning
        elif pretrain_stage == False:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            if load_pretrained_mdl_path == None:
                raise ValueError('Please set load_pretrained_mdl_path to load a pretrained models.')
            sd = torch.load(load_pretrained_mdl_path, map_location=device)
            # get the fshape and tshape, input_fdim and input_tdim in the pretraining stage
            try:
                p_fshape, p_tshape = sd['module.v.patch_embed.proj.weight'].shape[2], sd['module.v.patch_embed.proj.weight'].shape[3]
                p_input_fdim, p_input_tdim = sd['module.p_input_fdim'].item(), sd['module.p_input_tdim'].item()
            except:
                raise  ValueError('The model loaded is not from a torch.nn.Dataparallel object. Wrap it with torch.nn.Dataparallel and try again.')

            print('now load a SSL pretrained models from ' + load_pretrained_mdl_path)
            # during pretraining, fstride=fshape and tstride=tshape because no patch overlapping is used
            # here, input_fdim and input_tdim should be that used in pretraining, not that in the fine-tuning.
            # we need to know input_fdim and input_tdim to do positional embedding cut/interpolation.
            # generally it should be better to use same input_fdim during pretraining and finetuning, but input_tdim can be safely different
            audio_model = ASTModel(fstride=p_fshape, tstride=p_tshape, fshape=p_fshape, tshape=p_tshape,
                                   input_fdim=p_input_fdim, input_tdim=p_input_tdim, pretrain_stage=True, model_size=model_size)
            audio_model = torch.nn.DataParallel(audio_model)
            audio_model.load_state_dict(sd, strict=False)

            self.v = audio_model.module.v
            self.original_embedding_dim = self.v.pos_embed.shape[2]
            self.cls_token_num = audio_model.module.cls_token_num

            # mlp head for fine-tuning
            self.mlp_head = nn.Sequential(nn.LayerNorm(self.original_embedding_dim),
                                          nn.Linear(self.original_embedding_dim, label_dim))

            f_dim, t_dim = self.get_shape(fstride, tstride, input_fdim, input_tdim, fshape, tshape)
            # patch array dimension during pretraining
            p_f_dim, p_t_dim = audio_model.module.p_f_dim, audio_model.module.p_t_dim
            num_patches = f_dim * t_dim
            p_num_patches = p_f_dim * p_t_dim
            self.v.patch_embed.num_patches = num_patches
            print('fine-tuning patch split stride: frequncey={:d}, time={:d}'.format(fstride, tstride))
            print('fine-tuning number of patches={:d}'.format(num_patches))

            # patch shape should be same for pretraining and fine-tuning
            if fshape != p_fshape or tshape != p_tshape:
                raise ValueError('The patch shape of pretraining and fine-tuning is not consistant, pretraining: f={:d}, t={:d}, finetuning: f={:d}, t={:d}'.format(p_fshape, p_tshape, fshape, tshape))

            # patch split stride generally should be different for pretraining and fine-tuning, as patch split overlapping is only used in finetuning
            # during pretraining, p_fshape = p_fstride and p_tshape = p_tstride
            if fstride != p_fshape or tstride != p_tshape:
                # initialize a new patch embedding layer with desired new stride.
                new_proj = torch.nn.Conv2d(1, self.original_embedding_dim, kernel_size=(fshape, tshape), stride=(fstride, tstride))
                # but the weights of patch embedding layer is still got from the pretrained models
                new_proj.weight = torch.nn.Parameter(torch.sum(self.v.patch_embed.proj.weight, dim=1).unsqueeze(1))
                new_proj.bias = self.v.patch_embed.proj.bias
                self.v.patch_embed.proj = new_proj

            new_pos_embed = self.v.pos_embed[:, self.cls_token_num:, :].detach().reshape(1, p_num_patches, self.original_embedding_dim).transpose(1, 2).reshape(1, self.original_embedding_dim, p_f_dim, p_t_dim)
            # cut or interpolate the positional embedding
            if t_dim < p_t_dim:
                new_pos_embed = new_pos_embed[:, :, :, int(p_t_dim/2) - int(t_dim / 2): int(p_t_dim/2) - int(t_dim / 2) + t_dim]
            else:
                new_pos_embed = torch.nn.functional.interpolate(new_pos_embed, size=(8, t_dim), mode='bilinear')
            if f_dim < p_f_dim:
                new_pos_embed = new_pos_embed[:, :, int(p_f_dim/2) - int(f_dim / 2): int(p_f_dim/2) - int(f_dim / 2) + t_dim, :]
            else:
                new_pos_embed = torch.nn.functional.interpolate(new_pos_embed, size=(f_dim, t_dim), mode='bilinear')

            new_pos_embed = new_pos_embed.reshape(1, self.original_embedding_dim, num_patches).transpose(1, 2)
            self.v.pos_embed = nn.Parameter(torch.cat([self.v.pos_embed[:, :self.cls_token_num, :].detach(), new_pos_embed], dim=1))

    # get the shape of intermediate representation.
    def get_shape(self, fstride, tstride, input_fdim, input_tdim, fshape, tshape):
        test_input = torch.randn(1, 1, input_fdim, input_tdim)
        test_proj = nn.Conv2d(1, self.original_embedding_dim, kernel_size=(fshape, tshape), stride=(fstride, tstride))
        test_out = test_proj(test_input)
        f_dim = test_out.shape[2]
        t_dim = test_out.shape[3]
        return f_dim, t_dim


    def finetuningavgtok(self, x):
        B = x.shape[0]
        x = self.v.patch_embed(x)
        if self.cls_token_num == 2:
            cls_tokens = self.v.cls_token.expand(B, -1, -1)
            dist_token = self.v.dist_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, dist_token, x), dim=1)
        else:
            cls_tokens = self.v.cls_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.v.pos_embed
        x = self.v.pos_drop(x)

        for blk_id, blk in enumerate(self.v.blocks):
            x = blk(x)
        x = self.v.norm(x)

        # average output of all tokens except cls token(s)
        x = torch.mean(x[:, self.cls_token_num:, :], dim=1)
        x = self.mlp_head(x)
        return x

    def finetuningcls(self, x):
        B = x.shape[0]
        x = self.v.patch_embed(x)
        if self.cls_token_num == 2:
            cls_tokens = self.v.cls_token.expand(B, -1, -1)
            dist_token = self.v.dist_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, dist_token, x), dim=1)
        else:
            cls_tokens = self.v.cls_token.expand(B, -1, -1)
            x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.v.pos_embed
        x = self.v.pos_drop(x)

        for blk_id, blk in enumerate(self.v.blocks):
            x = blk(x)
        x = self.v.norm(x)

        # if models has two cls tokens (DEIT), average as the clip-level representation
        if self.cls_token_num == 2:
            x = (x[:, 0] + x[:, 1]) / 2
        else:
            x = x[:, 0]
        x = self.mlp_head(x)
        return x

    
    
    def forward(self, x, task, cluster=True, mask_patch=400):
        # expect input x = (batch_size, time_frame_num, frequency_bins), e.g., (12, 1024, 128)
        x = x.unsqueeze(1)
        x = x.transpose(2, 3)

        # finetuning (ft), use the mean of all token (patch) output as clip-level representation.
        # this is default for SSAST fine-tuning as during pretraining, supervision signal is given to each token, not the [cls] token
        if task == 'ft_avgtok':
            return self.finetuningavgtok(x)
        # alternatively, use the [cls] token output as clip-level representation.
        elif task == 'ft_cls':
            return self.finetuningcls(x)
        else:
            raise Exception('Task unrecognized.')


# Utils

In [ ]:
import numpy as np
from scipy import stats
from sklearn import metrics
import torch

def d_prime(auc):
    standard_normal = stats.norm()
    d_prime = standard_normal.ppf(auc) * np.sqrt(2.0)
    return d_prime

def calculate_stats(output, target, multi_label=False):
    """Calculate statistics including mAP, AUC, etc.
    Args:
      output: 2d array, (samples_num, classes_num)
      target: 2d array, (samples_num, classes_num)
    Returns:
      stats: list of statistic of each class.
    """

    classes_num = target.shape[-1]
    stats = []

    # Accuracy, only used for single-label classification such as esc-50, not for multiple label one such as AudioSet
    acc = metrics.accuracy_score(np.argmax(target, 1), np.argmax(output, 1))

    # Class-wise statistics
    if multi_label == True:
        for k in range(classes_num):
            if target[:, k].sum() / target.shape[0] == target[0][k]:
                continue

            # Average precision
            avg_precision = metrics.average_precision_score(
                target[:, k], output[:, k], average=None)

            # AUC
            auc = metrics.roc_auc_score(target[:, k], output[:, k], average=None)

            # Precisions, recalls
            (precisions, recalls, thresholds) = metrics.precision_recall_curve(
                target[:, k], output[:, k])

            # FPR, TPR
            (fpr, tpr, thresholds) = metrics.roc_curve(target[:, k], output[:, k])

            save_every_steps = 1000     # Sample statistics to reduce size
            dict = {'precisions': precisions[0::save_every_steps],
                    'recalls': recalls[0::save_every_steps],
                    'AP': avg_precision,
                    'fpr': fpr[0::save_every_steps],
                    'fnr': 1. - tpr[0::save_every_steps],
                    'auc': auc,
                    # note acc is not class-wise, this is just to keep consistent with other metrics
                    'acc': acc
                    }
            stats.append(dict)
    else:
        stats.append({'acc': acc})
    
    return stats

# train function

In [ ]:
import sys
import os
import datetime
# sys.path.append(os.path.dirname(os.path.dirname(sys.path[0])))
import time
import torch
from torch import nn
import numpy as np
from torch.cuda.amp import autocast,GradScaler

valid_loss_list = []
valid_acc_list = []

train_loss_list = []

def train(audio_model, train_loader, test_loader, args, pretrained_path=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('running on ' + str(device))
    torch.set_grad_enabled(True)

    # best_cum_mAP is checkpoint ensemble from the first epoch to the best epoch
    best_epoch, best_cum_epoch, best_mAP, best_acc, best_cum_mAP = 0, 0, -np.inf, -np.inf, -np.inf
    best_loss = 10000
    global_step, epoch = 0, 0
    start_time = time.time()

    if not isinstance(audio_model, nn.DataParallel):
        audio_model = nn.DataParallel(audio_model)
        
    if pretrained_path:
        checkpoint = torch.load(pretrained_path)
        audio_model.load_state_dict(checkpoint['model_state_dict'])
        # best_loss = checkpoint['best_loss']

    audio_model = audio_model.to(device)
    # Set up the optimizer
    trainables = [p for p in audio_model.parameters() if p.requires_grad]
    print('Total parameter number is : {:.3f} million'.format(sum(p.numel() for p in audio_model.parameters()) / 1e6))
    print('Total trainable parameter number is : {:.3f} million'.format(sum(p.numel() for p in trainables) / 1e6))

    # diff lr optimizer
    mlp_list = ['mlp_head.0.weight', 'mlp_head.0.bias', 'mlp_head.1.weight', 'mlp_head.1.bias']
    mlp_params = list(filter(lambda kv: kv[0] in mlp_list, audio_model.module.named_parameters()))
    base_params = list(filter(lambda kv: kv[0] not in mlp_list, audio_model.module.named_parameters()))
    mlp_params = [i[1] for i in mlp_params]
    base_params = [i[1] for i in base_params]
    # only finetuning small/tiny models on balanced audioset uses different learning rate for mlp head
    print('The mlp header uses {:d} x larger lr'.format(args["head_lr"]))
    if args["optimizer"] == 'adam':
        optimizer = torch.optim.Adam(
            [
                {'params': base_params, 'lr': args["lr"]},
                {'params': mlp_params, 'lr': args["lr"] * args["head_lr"]}
            ], weight_decay=5e-7, betas=(0.95, 0.999)
        )
    elif args["optimizer"] == "adamw":
        optimizer = torch.optim.AdamW(
            [
                {'params': base_params, 'lr': args["lr"]},
                {'params': mlp_params, 'lr': args["lr"] * args["head_lr"]}
            ], weight_decay=5e-7, amsgrad=True
        )
    else:
        optimizer = torch.optim.SGD(
            [
                {'params': base_params, 'lr': args["lr"]},
                {'params': mlp_params, 'lr': args["lr"] * args["head_lr"]}
            ], momentum=0.9, nesterov=True, weight_decay=5e-7
        )
    
    if pretrained_path:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
    
    mlp_lr = optimizer.param_groups[1]['lr']
    lr_list = [args["lr"], mlp_lr]

    print('Total mlp parameter number is : {:.3f} million'.format(sum(p.numel() for p in mlp_params) / 1e6))
    print('Total base parameter number is : {:.3f} million'.format(sum(p.numel() for p in base_params) / 1e6))

    if args["adaptschedule"] == True:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=args["lr_patience"], verbose=True)
        print('now use adaptive learning rate scheduler.')
    else:
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, list(range(args["lrscheduler_start"], 1000, args["lrscheduler_step"])),gamma=args["lrscheduler_decay"])
    main_metrics = args["metrics"]
    if args["loss"] == 'BCE':
        loss_fn = nn.BCEWithLogitsLoss()
    elif args["loss"] == 'CE':
        loss_fn = nn.CrossEntropyLoss()
    args["loss_fn"] = loss_fn

    
    epoch += 1

    print("current #steps=%s, #epochs=%s" % (global_step, epoch))
    print("start training...")
    result = np.zeros([args["n_epochs"], 10])
    audio_model.train()
    while epoch < args["n_epochs"] + 1:
        begin_time = time.time()
        audio_model.train()
        print('---------------')
        print(datetime.datetime.now())
        print("current #epochs=%s, #steps=%s" % (epoch, global_step))
        epoch_loss = []

        for i, (audio_input, labels) in enumerate(train_loader):

            B = audio_input.size(0)
            audio_input = audio_input.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            # first several steps for warm-up
            if global_step <= 1000 and global_step % 50 == 0 and args["warmup"] == True:
                for group_id, param_group in enumerate(optimizer.param_groups):
                    warm_lr = (global_step / 1000) * lr_list[group_id]
                    param_group['lr'] = warm_lr
                    print('warm-up learning rate is {:f}'.format(param_group['lr']))

            audio_output = audio_model(audio_input, args["task"])
            if isinstance(loss_fn, torch.nn.CrossEntropyLoss):
                loss = loss_fn(audio_output, torch.argmax(labels.long(), axis=1))
            else:
                loss = loss_fn(audio_output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print_step = global_step % args["n_print_steps"] == 0
            early_print_step = epoch == 0 and global_step % (args["n_print_steps"]/10) == 0
            print_step = print_step or early_print_step

            if print_step and global_step != 0:
                print(f'epoch {epoch}, loss {float(loss):f}')

            global_step += 1
            epoch_loss.append(float(loss))

        print('start validation')
        stats, valid_loss = validate(audio_model, test_loader, args, epoch)
        
        acc = stats[0]['acc']
        
#         if main_metrics == 'mAP':
#             mAP = np.mean([stat['AP'] for stat in stats])
#             mAUC = np.mean([stat['auc'] for stat in stats])
            
#             middle_ps = [stat['precisions'][int(len(stat['precisions'])/2)] for stat in stats]
#             middle_rs = [stat['recalls'][int(len(stat['recalls'])/2)] for stat in stats]
#             average_precision = np.mean(middle_ps)
#             average_recall = np.mean(middle_rs)
        
        valid_loss_list.append(valid_loss)
        valid_acc_list.append(acc)

        if main_metrics == 'mAP':
            print("mAP: {:.6f}".format(mAP))
        else:
            print("acc: {:.6f}".format(acc))
#         print("AUC: {:.6f}".format(mAUC))
#         print("Avg Precision: {:.6f}".format(average_precision))
#         print("Avg Recall: {:.6f}".format(average_recall))
#         print("d_prime: {:.6f}".format(d_prime(mAUC)))
#         print("valid_loss: {:.6f}".format(valid_loss))

        print('validation finished')

#         if mAP > best_mAP:
#             best_mAP = mAP
#             if main_metrics == 'mAP':
#                 best_epoch = epoch

        if acc > best_acc:
            best_acc = acc
            if main_metrics == 'acc':
                best_epoch = epoch
                
        epoch_loss_mean = np.mean(epoch_loss)
        print("epoch loss: {:.6f}".format(epoch_loss_mean))
        if epoch_loss_mean < best_loss:
            best_loss = epoch_loss_mean
            if main_metrics == 'loss':
                best_epoch = epoch
        
        if best_epoch == epoch:
            torch.save({
                'epoch': epoch,
                'model_state_dict': audio_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_loss': best_loss,
            }, './model-stride-10-adamw-scored-just-scored.pth')

        if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            print('adaptive learning rate scheduler step')
            scheduler.step(mAP)
        else:
            print('normal learning rate scheduler step')
            scheduler.step()

        print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))
        print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[1]['lr']))

        finish_time = time.time()
        print('epoch {:d} training time: {:.3f}'.format(epoch, finish_time-begin_time))

        epoch += 1

def validate(audio_model, val_loader, args, epoch):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if not isinstance(audio_model, nn.DataParallel):
        audio_model = nn.DataParallel(audio_model)
    audio_model = audio_model.to(device)
    # switch to evaluate mode
    audio_model.eval()

    A_predictions = []
    A_targets = []
    A_loss = []
    # args["loss_fn"] = nn.BCEWithLogitsLoss()
    with torch.no_grad():
        for i, (audio_input, labels) in enumerate(val_loader):
            audio_input = audio_input.to(device)

            # compute output
            audio_output = audio_model(audio_input, args["task"])
            audio_output_for_loss = audio_output
            if isinstance(args["loss_fn"], torch.nn.CrossEntropyLoss):
                audio_output = torch.nn.functional.softmax(audio_output, dim=-1)
            else:
                audio_output = torch.sigmoid(audio_output)
            predictions = audio_output.to('cpu').detach()

            A_predictions.append(predictions)
            A_targets.append(labels)

            # compute the loss
            labels = labels.to(device)
            if isinstance(args["loss_fn"], torch.nn.CrossEntropyLoss):
                loss = args["loss_fn"](audio_output_for_loss, torch.argmax(labels.long(), axis=1))
            else:
                loss = args["loss_fn"](audio_output_for_loss, labels)
            A_loss.append(loss.to('cpu').detach())

        audio_output = torch.cat(A_predictions)
        target = torch.cat(A_targets)
        loss = np.mean(A_loss)
        stats = calculate_stats(audio_output, target, False)

    return stats, loss

# Training

In [ ]:
args = {
    "exp_dir": "",
    "head_lr": 1,
    "lr": 0.0001,
    "lrscheduler_start": 7,
    "lrscheduler_step": 5,
    "lrscheduler_decay": 0.1,
    "metrics": ["mAP", "acc"][1],
    "loss": ["BCE", "CE"][0],
    "n_epochs": 50,
    "warmup": True,
    "task": ["ft_avgtok", "ft_cls"][1],
    "n_print_steps": 100,
    "wa": True,
    "wa_start": 1,
    "wa_znd": 15,
    "save_model": False,
    "adaptschedule": False,
    "optimizer": ["adam", "adamw", "sgd"][1]
}

audio_model = ASTModel(label_dim=len(labels),
                 fshape=16, tshape=16, fstride=10, tstride=10,
                 input_fdim=128, input_tdim=target_length, model_size='base',
                 pretrain_stage=False,
                 load_pretrained_mdl_path=pretrained_mdl_path)

if not isinstance(audio_model, torch.nn.DataParallel):
    audio_model = torch.nn.DataParallel(audio_model)

In [ ]:
# summary(audio_model, input_size=(batch_size, target_length, num_mel_bins), task=args["task"])

In [ ]:
# stats, valid_loss = validate(audio_model, test_dataloader, args, 1)

In [ ]:
# train(audio_model, train_dataloader, test_dataloader,  args)

# Submit

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import librosa
import math

def get_score(audio_model):
    test_audio_dir = '../input/birdclef-2022/test_soundscapes/'
    #test_audio_dir = '../input/birdclef-2022/train_audio/akiapo/'
    file_list = [f.split('.')[0] for f in sorted(os.listdir(test_audio_dir))]
    with open('../input/birdclef-2022/scored_birds.json') as sbfile:
        scored_birds = json.load(sbfile)
        threshold = 0.7
    pred = {'row_id': [], 'target': []}
    for afile in file_list:
        path = test_audio_dir + afile + '.ogg'
        
        waveform, sr = torchaudio.load(path)
        waveform = waveform - waveform.mean()
        
        length = waveform.shape[1] // sr
        chunks = [[] for i in range(math.ceil(length/5))] #length//5
        for i in range(len(chunks)):        
            chunk_end_time = (i + 1) * 5
            fbank = torchaudio.compliance.kaldi.fbank(waveform[:, i*5*sr:(i+1)*5*sr], htk_compat=True, sample_frequency=sr, use_energy=False,
                                                  window_type='hanning', num_mel_bins=num_mel_bins, dither=0.0, frame_shift=10)
            n_frames = fbank.shape[0]
            p = target_length - n_frames
            if p > 0:
                m = torch.nn.ZeroPad2d((0, 0, 0, p))
                fbank = m(fbank)
            elif p < 0:
                fbank = fbank[0:target_length, :]
            fbank = (fbank - mean) / (std * 2)
            aud = torch.unsqueeze(fbank, 0)
            with torch.no_grad():
                scores = audio_model(aud, args["task"])
            #scores = torch.nn.functional.softmax(scores, dim=-1)
            scores = torch.sigmoid(scores)
            
            for bird in scored_birds:
                idx = label2id[bird]
                score = scores[0][idx]
                row_id = afile + '_' + bird + '_' + str(chunk_end_time)
                pred['row_id'].append(row_id)
                pred['target'].append(True if score > threshold else False)
    results = pd.DataFrame(pred, columns = ['row_id', 'target'])
    return results

In [ ]:
def submit(pretrained_path=None):
    if pretrained_path:
        checkpoint = torch.load(pretrained_path)
        #checkpoint = torch.load(pretrained_path, map_location=torch.device('cpu'))
        audio_model.load_state_dict(checkpoint['model_state_dict'])
    audio_model.eval()
    return get_score(audio_model)

In [ ]:
results = submit("../input/birdclef2022-epoch10/model-stride-10-adamw-scored.pth")
# pd.options.display.max_rows = None
# print(results)
results.to_csv("submission.csv", index=False)

In [ ]:
#print(np.mean([t.cpu() for t in tresholds]))